In [104]:
import pandas as pd
import numpy as np

In [105]:
df = pd.read_csv('../metadata/relationships/predictions_0.3_0.55_avg.csv')

In [106]:
df

,ImageId,PredictionString
0,00000b4dcff7f799,/m/07r04 0.674956 0.0006 0.0836 0.6038 0.9099 ...
1,00001a21632de752,/m/03bt1vf 0.547448 0.4035 0.4621 0.4987 0.675...
2,0000d67245642c5f,/m/0d5gx 0.768605 0.0006 0.0000 0.9994 0.9981
3,0001244aa8ed3099,/m/07j7r 0.402949 0.0006 0.0000 0.9994 0.9381 ...
4,000172d1dd1adce0,/m/0d5gx 0.792184 0.1912 0.0000 0.8373 0.8764 ...
...,...,...
4995,0be7351210a9e935,/m/06msq 0.574559 0.2298 0.0447 0.9254 0.9981 ...
4996,0be79d4ed1a1c8c8,/m/019jd 0.418313 0.0006 0.0000 0.9994 0.7960 ...
4997,0be7e0abf48d0c8b,/m/0by6g 0.210599 0.0940 0.0416 0.3800 0.5235 ...
4998,0be83de07c560ba7,/m/06m11 0.163675 0.0545 0.2698 0.5598 0.9327 ...


## Splitting Objects

In [107]:
df['PredictionString'] = df['PredictionString'].str.split('/m')
df = (df
 .set_index(['ImageId'])['PredictionString']
 .apply(pd.Series)
 .stack()
 .reset_index()
 .rename(columns={0:'PredictionString'}))

In [108]:
df

,ImageId,level_1,PredictionString
0,00000b4dcff7f799,0,
1,00000b4dcff7f799,1,/07r04 0.674956 0.0006 0.0836 0.6038 0.9099
2,00000b4dcff7f799,2,/01j61q 0.530928 0.4445 0.1405 0.9148 0.7582
3,00000b4dcff7f799,3,/0fly7 0.466945 0.8770 0.5484 0.9447 0.7855
4,00000b4dcff7f799,4,/04yx4 0.425865 0.7446 0.3625 0.8726 0.8978
...,...,...,...
57558,0be83de07c560ba7,2,/0138tl 0.150178 0.4906 0.0054 0.9987 0.7913
57559,0be83de07c560ba7,3,/0cxn2 0.197302 0.0538 0.2652 0.5577 0.9342
57560,0be83de07c560ba7,4,/0cxn2 0.159126 0.5013 0.0134 1.0000 0.7989
57561,0be88e9736da69b3,0,


## Deleting NaN values

In [109]:
df['PredictionString'].replace('', np.nan, inplace=True)
df.dropna(subset=['PredictionString'], inplace=True)

In [110]:
df

,ImageId,level_1,PredictionString
1,00000b4dcff7f799,1,/07r04 0.674956 0.0006 0.0836 0.6038 0.9099
2,00000b4dcff7f799,2,/01j61q 0.530928 0.4445 0.1405 0.9148 0.7582
3,00000b4dcff7f799,3,/0fly7 0.466945 0.8770 0.5484 0.9447 0.7855
4,00000b4dcff7f799,4,/04yx4 0.425865 0.7446 0.3625 0.8726 0.8978
5,00000b4dcff7f799,5,/01g317 0.349475 0.7370 0.3650 0.8683 0.8981
...,...,...,...
57557,0be83de07c560ba7,1,/06m11 0.163675 0.0545 0.2698 0.5598 0.9327
57558,0be83de07c560ba7,2,/0138tl 0.150178 0.4906 0.0054 0.9987 0.7913
57559,0be83de07c560ba7,3,/0cxn2 0.197302 0.0538 0.2652 0.5577 0.9342
57560,0be83de07c560ba7,4,/0cxn2 0.159126 0.5013 0.0134 1.0000 0.7989


In [111]:
df['PredictionString'] = '/m' + df['PredictionString'].astype(str)

In [112]:
df

,ImageId,level_1,PredictionString
1,00000b4dcff7f799,1,/m/07r04 0.674956 0.0006 0.0836 0.6038 0.9099
2,00000b4dcff7f799,2,/m/01j61q 0.530928 0.4445 0.1405 0.9148 0.7582
3,00000b4dcff7f799,3,/m/0fly7 0.466945 0.8770 0.5484 0.9447 0.7855
4,00000b4dcff7f799,4,/m/04yx4 0.425865 0.7446 0.3625 0.8726 0.8978
5,00000b4dcff7f799,5,/m/01g317 0.349475 0.7370 0.3650 0.8683 0.8981
...,...,...,...
57557,0be83de07c560ba7,1,/m/06m11 0.163675 0.0545 0.2698 0.5598 0.9327
57558,0be83de07c560ba7,2,/m/0138tl 0.150178 0.4906 0.0054 0.9987 0.7913
57559,0be83de07c560ba7,3,/m/0cxn2 0.197302 0.0538 0.2652 0.5577 0.9342
57560,0be83de07c560ba7,4,/m/0cxn2 0.159126 0.5013 0.0134 1.0000 0.7989


## Deviding it to 6 columns

In [113]:
df[['LabelName','Score', 'XMin1', 'YMin1', 'XMax1', 'YMax1', 'space']] = df['PredictionString'].str.split(' ', expand=True)

In [114]:
df = df[['ImageId', 'LabelName', 'Score', 'XMin1', 'YMin1', 'XMax1', 'YMax1']]
df

,ImageId,LabelName,Score,XMin1,YMin1,XMax1,YMax1
1,00000b4dcff7f799,/m/07r04,0.674956,0.0006,0.0836,0.6038,0.9099
2,00000b4dcff7f799,/m/01j61q,0.530928,0.4445,0.1405,0.9148,0.7582
3,00000b4dcff7f799,/m/0fly7,0.466945,0.8770,0.5484,0.9447,0.7855
4,00000b4dcff7f799,/m/04yx4,0.425865,0.7446,0.3625,0.8726,0.8978
5,00000b4dcff7f799,/m/01g317,0.349475,0.7370,0.3650,0.8683,0.8981
...,...,...,...,...,...,...,...
57557,0be83de07c560ba7,/m/06m11,0.163675,0.0545,0.2698,0.5598,0.9327
57558,0be83de07c560ba7,/m/0138tl,0.150178,0.4906,0.0054,0.9987,0.7913
57559,0be83de07c560ba7,/m/0cxn2,0.197302,0.0538,0.2652,0.5577,0.9342
57560,0be83de07c560ba7,/m/0cxn2,0.159126,0.5013,0.0134,1.0000,0.7989


## Deleting objects under 0.5 Score

In [117]:
df = df[df['Score'].astype('float') >= 0.5]
df

,ImageId,LabelName,Score,XMin1,YMin1,XMax1,YMax1
1,00000b4dcff7f799,/m/07r04,0.674956,0.0006,0.0836,0.6038,0.9099
2,00000b4dcff7f799,/m/01j61q,0.530928,0.4445,0.1405,0.9148,0.7582
13,00001a21632de752,/m/03bt1vf,0.547448,0.4035,0.4621,0.4987,0.6753
14,00001a21632de752,/m/03bt1vf,0.521230,0.2522,0.5998,0.5313,0.9981
15,00001a21632de752,/m/03bt1vf,0.502450,0.4494,0.4837,0.6137,0.8748
...,...,...,...,...,...,...,...
57529,0be7304b308812b2,/m/01rkbr,0.579395,0.5113,0.6947,0.6225,0.9981
57530,0be7304b308812b2,/m/04yx4,0.573280,0.2728,0.1418,0.9081,0.9981
57531,0be7304b308812b2,/m/0dzct,0.561340,0.4776,0.3051,0.6889,0.6950
57539,0be7351210a9e935,/m/06msq,0.574559,0.2298,0.0447,0.9254,0.9981


## Replacing LabelName

In [118]:
desc = pd.read_csv('../metadata/relationships/class-descriptions-boxable.csv')

In [119]:
desc_labels = dict(zip(desc.LabelID, desc.LabelName))
df['LabelName'] = df.LabelName.map(desc_labels)

In [120]:
df

,ImageId,LabelName,Score,XMin1,YMin1,XMax1,YMax1
1,00000b4dcff7f799,Truck,0.674956,0.0006,0.0836,0.6038,0.9099
2,00000b4dcff7f799,Tent,0.530928,0.4445,0.1405,0.9148,0.7582
13,00001a21632de752,Woman,0.547448,0.4035,0.4621,0.4987,0.6753
14,00001a21632de752,Woman,0.521230,0.2522,0.5998,0.5313,0.9981
15,00001a21632de752,Woman,0.502450,0.4494,0.4837,0.6137,0.8748
...,...,...,...,...,...,...,...
57529,0be7304b308812b2,Tie,0.579395,0.5113,0.6947,0.6225,0.9981
57530,0be7304b308812b2,Man,0.573280,0.2728,0.1418,0.9081,0.9981
57531,0be7304b308812b2,Human face,0.561340,0.4776,0.3051,0.6889,0.6950
57539,0be7351210a9e935,Sculpture,0.574559,0.2298,0.0447,0.9254,0.9981


## Deleteing whats not in  lables

In [121]:
labels = ['Bench', 'Wok', 'Parachute', 'Ambulance', 'Fox', 'Tablet computer', 'Picnic basket',
          'Dinosaur', 'Swim cap', 'Submarine sandwich', 'Lavender (Plant)', 'Hammer', 'Croissant',
          'Sunglasses', 'Taco', 'Cookie', 'Knife', 'Lizard', 'Pear', 'Wheelchair', 'Monkey', 'Tortoise',
          'Unicycle', 'High heels', 'Belt', 'Candy', 'Common sunflower', 'Tiara', 'Billiard table', 'Potato',
          'Handbag', 'Rifle', 'Salad', 'French fries', 'Cutting board', 'Tennis racket', 'Train', 'Pomegranate',
          'Balance beam', 'Motorcycle', 'Book', 'Kitchen knife', 'Sun hat', 'Racket', 'Horse', 'Sea lion',
          'Coffee table', 'Tripod', 'Table tennis racket', 'Axe', 'Bed', 'Cart', 'Saxophone', 'Baseball bat',
          'Sword', 'Dog bed', 'Broccoli', 'Doughnut', 'Earrings', 'Strawberry', 'Bottle', 'Whisk', 'Scarf',
          'Horizontal bar', 'Adhesive tape', 'Surfboard', 'Van', 'Pineapple', 'Oven', 'Sewing machine',
          'Baseball glove', 'Boot', 'Studio couch', 'Jet ski', 'Platter', 'Coconut', 'Toilet paper', 'Apple',
          'Banjo', 'Stool', 'Flowerpot', 'Balloon', 'Cabbage', 'Milk', 'Artichoke', 'Watermelon', 'Tin can',
          'Guitar', 'Golf ball', 'Scissors', 'Coffee', 'Egg (Food)', 'Beer', 'Crocodile', 'Skunk', 'Snowmobile',
          'Ladder', 'Garden Asparagus', 'Microwave oven', 'Taxi', 'Cowboy hat', 'Crown', 'Football', 'Airplane',
          'Shotgun', 'Jaguar (Animal)', 'Tennis ball', 'Shark', 'Car', 'Snowplow', 'Tree', 'Tent', 'French horn',
          'Bagel', 'Plate', 'Treadmill', 'Wine', 'Lemon', 'Camera', 'Pizza', 'Tree house', 'Infant bed', 'Box',
          'Bicycle helmet', 'Trumpet', 'Countertop', 'Snake', 'Cantaloupe', 'Palm tree', 'Flying disc', 'Honeycomb',
          'Mango', 'Woman', 'Banana', 'Fedora', 'Houseplant', 'Man', 'Carrot', 'Pen', 'Pancake', 'Table', 'Necklace',
          'Brown bear', 'Frying pan', 'Harpsichord', 'Crab', 'Trombone', 'Canoe', 'Barrel', 'Mixing bowl', 'Mushroom',
          'Elephant', 'Tea', 'Stethoscope', 'Dumbbell', 'Washing machine', 'Punching bag', 'Cat furniture', 'Tank', 'Bicycle',
          'Mug', 'Limousine', 'Whale', 'Harbor seal', 'Cello', 'Dolphin', 'Lantern', 'Watch', 'Sombrero', 'Grape', 'Torch',
          'Doll', 'Cake', 'Snowboard', 'Pumpkin', 'Teddy bear', 'Stretcher', 'Popcorn', 'Peach', 'Rose', 'Gondola', 'Paddle',
          'Cheese', 'Candle', 'Coffee cup', 'Rays and skates', 'Ski', 'Waste container', 'Glasses', 'Violin', 'Cat', 'Bread',
          'Grapefruit', 'Teapot', 'Piano', 'Bow and arrow', 'Saucer', 'Cucumber', 'Tomato', 'Plastic bag', 'Cake stand', 'Rugby ball',
          'Spoon', 'Ice cream', 'Sofa bed', 'Drawer', 'Hamburger', 'Corded phone', 'Christmas tree', 'Suitcase', 'Flute', 'Bus', 'Bowl',
          'Indoor rower', 'Zucchini', 'Wine glass', 'Desk', 'Drum', 'Panda', 'Closet', 'Bicycle wheel', 'Kite', 'Football helmet', 'Fork',
          'Pitcher (Container)', 'Truck', 'Serving tray', 'Golf cart', 'Radish', 'Harp', 'Seahorse', 'Muffin', 'Sandal', 'Organ (Musical Instrument)',
          'Microphone', 'Chopsticks', 'Binoculars', 'Orange', 'Cannon', 'Hamster', 'Boy', 'Lobster', 'Musical keyboard', 'Bell pepper', 'Cricket ball',
          'Common fig', 'Cupboard', 'Drinking straw', 'Helicopter', 'Burrito', 'Shrimp', 'Girl', 'Sea turtle', 'Segway', 'Skateboard', 'Personal flotation device',
          'Kitchen & dining room table', 'Backpack', 'Polar bear', 'Briefcase', 'Oboe', 'Boat', 'Mobile phone', 'Roller skates', 'Juice', 'Bowling equipment',
          'Hiking equipment', 'Chair', 'Cocktail', 'Alpaca', 'Volleyball (Ball)', 'Goggles', 'Lynx', 'Tart', 'Jug', 'Stationary bicycle', 'Waffle',
          'Pasta', 'Accordion', 'Whiteboard', 'Maple', 'Sushi', 'Oyster', 'Dog', 'Handgun', 'Pretzel', 'Lily', 'Loveseat']

In [122]:
df = df[df.LabelName.isin(labels)]

In [123]:
df

,ImageId,LabelName,Score,XMin1,YMin1,XMax1,YMax1
1,00000b4dcff7f799,Truck,0.674956,0.0006,0.0836,0.6038,0.9099
2,00000b4dcff7f799,Tent,0.530928,0.4445,0.1405,0.9148,0.7582
13,00001a21632de752,Woman,0.547448,0.4035,0.4621,0.4987,0.6753
14,00001a21632de752,Woman,0.521230,0.2522,0.5998,0.5313,0.9981
15,00001a21632de752,Woman,0.502450,0.4494,0.4837,0.6137,0.8748
...,...,...,...,...,...,...,...
57517,0be6543c82380757,Snake,0.639521,0.0308,0.0924,0.5488,0.8383
57520,0be6bfa00f069a68,Train,0.508139,0.4096,0.0121,0.9475,0.9942
57521,0be6bfa00f069a68,Boat,0.519122,0.4126,0.0318,0.9541,0.9909
57530,0be7304b308812b2,Man,0.573280,0.2728,0.1418,0.9081,0.9981
